In [1]:
import numpy as np
import math
import cv2
import tdmclient.notebook


In [2]:
await tdmclient.notebook.start(tdm_addr='172.25.112.1', tdm_port=8596)

In [4]:
# await tdmclient.notebook.stop()

In [3]:
%%run_aseba
onevent run
    motor.target.left = 300
    motor.target.right = 300

onevent turnleft
    motor.target.left = -300
    motor.target.right = 300

onevent turnright
    motor.target.left = 300
    motor.target.right = -300

onevent stop
    motor.target.left = 0
    motor.target.right = 0

Exception: run is not a known event, do you mean rc5?

Define possible objects and marker colors

In [ ]:
robot1 = {"front":1, "back":2, "f_rgb": [136, 0, 20], "b_rgb": [3, 161, 234]}
robot2 = {"front":3, "back":4, "f_rgb": [23, 255, 22], "b_rgb": [163, 73, 164]}
ball = {"center":5, "rgb": [254, 242, 0]}

Define what robots are in the game

In [ ]:
robots = {
    "robot1": robot1,
    "robot2": robot2
}
color_tolerance = 20

In [ ]:
cap = cv2.VideoCapture(0)